In [ ]:
<!DOCTYPE html>
<html lang="fr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Historique des Prix - {{ product_name }}</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css" rel="stylesheet">
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <style>
        .price-card {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
        }

        .stats-card {
            border: none;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            transition: transform 0.2s;
        }

        .stats-card:hover {
            transform: translateY(-2px);
        }

        .chart-container {
            background: white;
            border-radius: 10px;
            padding: 20px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }

        .period-btn {
            border-radius: 20px;
            margin: 0 5px;
        }

        .period-btn.active {
            background: #667eea;
            border-color: #667eea;
        }

        .price-up {
            color: #28a745;
        }

        .price-down {
            color: #dc3545;
        }

        .alert-badge {
            position: absolute;
            top: -5px;
            right: -5px;
        }

        .export-btn {
            background: linear-gradient(135deg, #28a745 0%, #20c997 100%);
            border: none;
            color: white;
        }

        .back-btn {
            background: linear-gradient(135deg, #6c757d 0%, #495057 100%);
            border: none;
            color: white;
        }
    </style>
</head>
<body class="bg-light">
    <!-- Navigation -->
    <nav class="navbar navbar-expand-lg navbar-dark bg-primary">
        <div class="container">
            <a class="navbar-brand" href="/">
                <i class="fas fa-chart-line me-2"></i>Prix Tracker
            </a>
            <nav aria-label="breadcrumb">
                <ol class="breadcrumb mb-0">
                    <li class="breadcrumb-item"><a href="/" class="text-white">Accueil</a></li>
                    <li class="breadcrumb-item"><a href="/products" class="text-white">Produits</a></li>
                    <li class="breadcrumb-item active text-light">Historique</li>
                </ol>
            </nav>
        </div>
    </nav>

    <div class="container mt-4">
        <!-- En-tête produit -->
        <div class="row mb-4">
            <div class="col-12">
                <div class="card price-card">
                    <div class="card-body">
                        <div class="row align-items-center">
                            <div class="col-md-8">
                                <h2 class="card-title mb-1">{{ product_name }}</h2>
                                <p class="card-text opacity-75">{{ product_description }}</p>
                            </div>
                            <div class="col-md-4 text-end">
                                <h3 class="mb-0">{{ current_price }}€</h3>
                                <small class="price-change {{ 'price-up' if price_change > 0 else 'price-down' }}">
                                    <i class="fas {{ 'fa-arrow-up' if price_change > 0 else 'fa-arrow-down' }}"></i>
                                    {{ price_change }}% (24h)
                                </small>
                            </div>
                        </div>
                    </div>
                </div>
            </div>
        </div>

        <!-- Contrôles de période -->
        <div class="row mb-4">
            <div class="col-12 text-center">
                <div class="btn-group" role="group">
                    <button type="button" class="btn btn-outline-primary period-btn active" data-period="7">7 jours</button>
                    <button type="button" class="btn btn-outline-primary period-btn" data-period="30">30 jours</button>
                    <button type="button" class="btn btn-outline-primary period-btn" data-period="90">90 jours</button>
                    <button type="button" class="btn btn-outline-primary period-btn" data-period="365">1 an</button>
                </div>
            </div>
        </div>

        <!-- Graphique -->
        <div class="row mb-4">
            <div class="col-12">
                <div class="chart-container">
                    <canvas id="priceChart" height="100"></canvas>
                </div>
            </div>
        </div>

        <!-- Statistiques -->
        <div class="row mb-4">
            <div class="col-md-3 mb-3">
                <div class="card stats-card text-center">
                    <div class="card-body">
                        <i class="fas fa-arrow-up text-success fa-2x mb-2"></i>
                        <h5 class="card-title text-success">Prix Max</h5>
                        <h4 class="mb-0">{{ max_price }}€</h4>
                        <small class="text-muted">{{ max_price_date }}</small>
                    </div>
                </div>
            </div>
            <div class="col-md-3 mb-3">
                <div class="card stats-card text-center">
                    <div class="card-body">
                        <i class="fas fa-arrow-down text-danger fa-2x mb-2"></i>
                        <h5 class="card-title text-danger">Prix Min</h5>
                        <h4 class="mb-0">{{ min_price }}€</h4>
                        <small class="text-muted">{{ min_price_date }}</small>
                    </div>
                </div>
            </div>
            <div class="col-md-3 mb-3">
                <div class="card stats-card text-center">
                    <div class="card-body">
                        <i class="fas fa-chart-bar text-info fa-2x mb-2"></i>
                        <h5 class="card-title text-info">Prix Moyen</h5>
                        <h4 class="mb-0">{{ avg_price }}€</h4>
                        <small class="text-muted">Sur la période</small>
                    </div>
                </div>
            </div>
            <div class="col-md-3 mb-3">
                <div class="card stats-card text-center position-relative">
                    <div class="card-body">
                        <i class="fas fa-bell text-warning fa-2x mb-2"></i>
                        <h5 class="card-title text-warning">Alertes</h5>
                        <h4 class="mb-0">{{ active_alerts }}</h4>
                        <small class="text-muted">Actives</small>
                        {% if new_alerts > 0 %}
                        <span class="badge bg-danger alert-badge">{{ new_alerts }}</span>
                        {% endif %}
                    </div>
                </div>
            </div>
        </div>

        <!-- Tableau d'historique -->
        <div class="row mb-4">
            <div class="col-12">
                <div class="card">
                    <div class="card-header d-flex justify-content-between align-items-center">
                        <h5 class="mb-0">Historique des Prix</h5>
                        <div class="d-flex gap-2">
                            <input type="date" class="form-control form-control-sm" id="dateFrom" placeholder="Du">
                            <input type="date" class="form-control form-control-sm" id="dateTo" placeholder="Au">
                            <button class="btn btn-sm btn-outline-primary" id="filterBtn">
                                <i class="fas fa-filter"></i> Filtrer
                            </button>
                        </div>
                    </div>
                    <div class="card-body p-0">
                        <div class="table-responsive">
                            <table class="table table-hover mb-0" id="priceTable">
                                <thead class="table-light">
                                    <tr>
                                        <th>Date</th>
                                        <th>Prix</th>
                                        <th>Variation</th>
                                        <th>Source</th>
                                        <th>Actions</th>
                                    </tr>
                                </thead>
                                <tbody>
                                    {% for record in price_history %}
